In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#print('OpenCV Version (should be 3.1.0, with nonfree packages installed, for this tutorial):')
print(cv2.__version__)
import PIL
from PIL import Image, ImageOps
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from PIL import ImageFilter
from skimage.restoration import estimate_sigma, denoise_nl_means
from skimage.filters import gaussian
from skimage import exposure
from skimage import img_as_ubyte, img_as_float
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

3.3.1


# Some links...

https://www.pyimagesearch.com/2014/05/26/opencv-python-k-means-color-clustering/  
http://charlesleifer.com/blog/using-python-and-k-means-to-find-the-dominant-colors-in-images/  
https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_ml/py_kmeans/py_kmeans_opencv/py_kmeans_opencv.html

# Dataset loading

In [2]:
dataset = pd.read_csv('Z1-picture_cropping.csv', index_col=0, dtype={'foldername':'object'})
dataset.head()

,breed,file,xmin,ymin,xmax,ymax,foldername,filename,dog_number,prepfilename,xdelta,ydelta
n02085620_10074,Chihuahua,n02085620_10074,25.0,10.0,276.0,498.0,02085620,n02085620_10074,1,n02085620_10074-Chihuahua_cr1,251.0,488.0
n02085620_10131,Chihuahua,n02085620_10131,49.0,9.0,393.0,493.0,02085620,n02085620_10131,1,n02085620_10131-Chihuahua_cr1,344.0,484.0
n02085620_10621,Chihuahua,n02085620_10621,142.0,43.0,335.0,250.0,02085620,n02085620_10621,1,n02085620_10621-Chihuahua_cr1,193.0,207.0
n02085620_1073,Chihuahua,n02085620_1073,0.0,27.0,312.0,498.0,02085620,n02085620_1073,1,n02085620_1073-Chihuahua_cr1,312.0,471.0
n02085620_10976,Chihuahua,n02085620_10976,90.0,104.0,242.0,452.0,02085620,n02085620_10976,1,n02085620_10976-Chihuahua_cr1,152.0,348.0


In [3]:
le = preprocessing.LabelEncoder()
le.fit(dataset['breed'])
print(le.classes_)
dataset['breed_code'] = le.transform(dataset['breed'])
dataset.head()

['Afghan_hound' 'African_hunting_dog' 'Airedale'
 'American_Staffordshire_terrier' 'Appenzeller' 'Australian_terrier'
 'Bedlington_terrier' 'Bernese_mountain_dog' 'Blenheim_spaniel'
 'Border_collie' 'Border_terrier' 'Boston_bull' 'Bouvier_des_Flandres'
 'Brabancon_griffon' 'Brittany_spaniel' 'Cardigan'
 'Chesapeake_Bay_retriever' 'Chihuahua' 'Dandie_Dinmont' 'Doberman'
 'English_foxhound' 'English_setter' 'English_springer' 'EntleBucher'
 'Eskimo_dog' 'French_bulldog' 'German_shepherd'
 'German_short-haired_pointer' 'Gordon_setter' 'Great_Dane'
 'Great_Pyrenees' 'Greater_Swiss_Mountain_dog' 'Ibizan_hound'
 'Irish_setter' 'Irish_terrier' 'Irish_water_spaniel' 'Irish_wolfhound'
 'Italian_greyhound' 'Japanese_spaniel' 'Kerry_blue_terrier'
 'Labrador_retriever' 'Lakeland_terrier' 'Leonberg' 'Lhasa' 'Maltese_dog'
 'Mexican_hairless' 'Newfoundland' 'Norfolk_terrier' 'Norwegian_elkhound'
 'Norwich_terrier' 'Old_English_sheepdog' 'Pekinese' 'Pembroke'
 'Pomeranian' 'Rhodesian_ridgeback' 'Rottw

,breed,file,xmin,ymin,xmax,ymax,foldername,filename,dog_number,prepfilename,xdelta,ydelta,breed_code
n02085620_10074,Chihuahua,n02085620_10074,25.0,10.0,276.0,498.0,02085620,n02085620_10074,1,n02085620_10074-Chihuahua_cr1,251.0,488.0,17
n02085620_10131,Chihuahua,n02085620_10131,49.0,9.0,393.0,493.0,02085620,n02085620_10131,1,n02085620_10131-Chihuahua_cr1,344.0,484.0,17
n02085620_10621,Chihuahua,n02085620_10621,142.0,43.0,335.0,250.0,02085620,n02085620_10621,1,n02085620_10621-Chihuahua_cr1,193.0,207.0,17
n02085620_1073,Chihuahua,n02085620_1073,0.0,27.0,312.0,498.0,02085620,n02085620_1073,1,n02085620_1073-Chihuahua_cr1,312.0,471.0,17
n02085620_10976,Chihuahua,n02085620_10976,90.0,104.0,242.0,452.0,02085620,n02085620_10976,1,n02085620_10976-Chihuahua_cr1,152.0,348.0,17


In [16]:
subset = dataset[dataset['breed_code'] <31]
subset.shape

(20580, 13)

In [17]:
subset.head()

,breed,file,xmin,ymin,xmax,ymax,foldername,filename,dog_number,prepfilename,xdelta,ydelta,breed_code
n02085620_10074,Chihuahua,n02085620_10074,25.0,10.0,276.0,498.0,02085620,n02085620_10074,1,n02085620_10074-Chihuahua_cr1,251.0,488.0,17
n02085620_10131,Chihuahua,n02085620_10131,49.0,9.0,393.0,493.0,02085620,n02085620_10131,1,n02085620_10131-Chihuahua_cr1,344.0,484.0,17
n02085620_10621,Chihuahua,n02085620_10621,142.0,43.0,335.0,250.0,02085620,n02085620_10621,1,n02085620_10621-Chihuahua_cr1,193.0,207.0,17
n02085620_1073,Chihuahua,n02085620_1073,0.0,27.0,312.0,498.0,02085620,n02085620_1073,1,n02085620_1073-Chihuahua_cr1,312.0,471.0,17
n02085620_10976,Chihuahua,n02085620_10976,90.0,104.0,242.0,452.0,02085620,n02085620_10976,1,n02085620_10976-Chihuahua_cr1,152.0,348.0,17


# Picture pre-loader

In [18]:
def pictureloader(picture):
    patch_kw = dict(patch_size=5,      # 5x5 patches
                        patch_distance=6,  # 13x13 search area
                        multichannel=True)
    
    xmax = picture.xmax
    ymax = picture.ymax
    xmin = picture.xmin
    ymin = picture.ymin
    foldername = picture.foldername
    breed = picture.breed
    filename = picture.filename
    prepfilename = picture.prepfilename
        
    #Image opening
    img = Image.open('../Images/n{}-{}/{}.jpg'.format(foldername, breed, filename))
            
    #Image cropping
    area = (xmin, ymin, xmax, ymax)
    img = img.crop(area)
            
    #Image equalization
    img = exposure.equalize_hist(np.array(img))
            
    #Gaussian blur fliter
    sigma_est = np.mean(estimate_sigma(np.array(img), multichannel=True, average_sigmas=False))
     
    #Non linear mean
    img = denoise_nl_means(img, h=1.5*sigma_est, fast_mode=True, **patch_kw)
    #print(img)
    return list(img)

subset['Picture'] = subset.apply(pictureloader, axis=1).apply(np.array)

/Users/cricket/anaconda3/lib/python3.6/site-packages/skimage/exposure/exposure.py:63: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  warn("This might be a color image. The histogram will be "


In [19]:
subset.to_csv('Z2 - dogs-breed-sample.csv')

In [20]:
subset['Picture'].map(lambda x: x.shape)

n02085620_10074     (488, 251, 3)
n02085620_10131     (484, 344, 3)
n02085620_10621     (207, 193, 3)
n02085620_1073      (471, 312, 3)
n02085620_10976     (348, 152, 3)
n02085620_11140     (215, 434, 3)
n02085620_11238     (192, 106, 3)
n02085620_11258     (345, 287, 3)
n02085620_11337     (142, 170, 3)
n02085620_11477     (364, 241, 3)
n02085620_1152        (56, 46, 3)
n02085620_11696     (516, 485, 3)
n02085620_11818     (253, 181, 3)
n02085620_11948     (420, 461, 3)
n02085620_1205      (239, 204, 3)
n02085620_12101     (248, 345, 3)
n02085620_12334      (144, 91, 3)
n02085620_1235       (188, 81, 3)
n02085620_1271      (347, 305, 3)
n02085620_12718     (445, 278, 3)
n02085620_1298      (136, 287, 3)
n02085620_13151     (304, 203, 3)
n02085620_1321      (272, 137, 3)
n02085620_13383     (256, 178, 3)
n02085620_1346      (308, 431, 3)
n02085620_13964     (362, 277, 3)
n02085620_14252     (138, 180, 3)
n02085620_14413     (727, 404, 3)
n02085620_14516     (476, 436, 3)
n02085620_1455

In [21]:
X_train, X_test, y_train, y_test = train_test_split(subset, subset['breed_code'], test_size=0.33, random_state=0, stratify=subset['breed_code'])

# Class creation

In [24]:
class SiftBOW(BaseEstimator, TransformerMixin):

    def __init__(self, **params):        
        self.kmeans_params = {'n_clusters':50, 'verbose':0, 'random_state':0, 'max_iter': 300}
        self.sift_params = {'nfeatures':0}
        self.pca_params = {'n_components':10, 'random_state':0}
        self.cvect_params = {'ngram_range':(1,1), 'min_df':1, 'max_df':1.}
        self.color_params = {'n_clusters':30, 'random_state':0, 'max_iter': 300}
        self.pics_params = {'save_pics':True}
        
        self.clf_params = { 'kmean_sample_size':10000, 'apply_pca':True}
        
        self.params = {**{'kmeans_'+k:v for k,v in self.kmeans_params.items()},
                       **{'sift_'+k:v for k,v in self.sift_params.items()},
                       **{'cvect_'+k:v for k,v in self.cvect_params.items()},
                       **{'clf_'+k:v for k,v in self.clf_params.items()},
                       **{'pca_'+k:v for k,v in self.pca_params.items()},
                       **{'color_'+k:v for k,v in self.color_params.items()},
                       **{'pics_'+k:v for k,v in self.pics_params.items()},
                       **params}
        
        self.update_params()
        self.kmeans = KMeans(**self.kmeans_params)
        self.scaler = StandardScaler()
        self.color_scaler = StandardScaler()
        self.cvect = CountVectorizer(**self.cvect_params, analyzer='word', tokenizer=None)
        self.sift = cv2.xfeatures2d.SIFT_create(**self.sift_params)
        #self.sift = cv2.ORB()
        self.pca = PCA(**self.pca_params)
        self.color_kmeans = KMeans(**self.color_params)
        
    def get_params(self, deep=True):
        return self.params
    
    def set_params(self, **params):
        self.params = {**self.params, **params}
        self.update_params()
        return self
        
    def update_params(self):
        
        self.kmeans_params = {k[7:]:v for k,v in self.params.items() if k.startswith('kmeans_')}
        self.sift_params = {k[5:]:v for k,v in self.params.items() if k.startswith('sift_')}
        self.pics_params = {k[5:]:v for k,v in self.params.items() if k.startswith('pics_')}
        self.color_params = {k[6:]:v for k,v in self.params.items() if k.startswith('color_')}
        self.cvect_params = {k[6:]:v for k,v in self.params.items() if k.startswith('cvect_')}
        self.pca_params = {k[4:]:v for k,v in self.params.items() if k.startswith('pca_')}
        self.clf_params = {k[4:]:v for k,v in self.params.items() if k.startswith('clf_')}

        return self

    
    def fit_bow(self, keydescriptors):
        self.cvect.fit(keydescriptors)
        print('BOW fitted')
        return self
    
    def fit_colors(self, colors, samplesize=10000):
        colorlist = colors.reshape(-1,3)
        indices = np.random.randint(0, colorlist.shape[0], samplesize)
        Xsample = colors[indices]
        Xscaled = self.color_scaler.fit_transform(Xsample)
        self.color_kmeans.fit(Xscaled)
        print('Color dominance fitted')
        return self
    
    def transform_bow(self, keydescriptors):
        return self.cvect.transform(keydescriptors)
    
    def fit(self, X, y= None):
        sift_desc, color_dominance = self.load_pics(X, y)
        self.fit_sift(np.concatenate(sift_desc))
        self.fit_colors(np.concatenate(color_dominance))
        
        bow = self.compute_data(sift_desc, color_dominance)
        self.fit_bow(bow)
        Xbow = self.transform_bow(bow)
        return self
    
    def color_domincance(self, picture, n_cluster=10):
        #Converts to openCV
        imcv = cv2.cvtColor(np.array(picture), cv2.COLOR_RGB2BGR)
        
        #Flatten all the pixels
        Z = imcv.reshape((-1,3))

        # convert to np.float32
        Z = np.float32(Z)

        # define criteria, number of clusters(K) and apply kmeans()
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        K = n_cluster
        ret, label, color_dominance = cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
        return color_dominance
    
    def load_pics(self, X, y=None):
        sift_desc = []
        colors = []
        filenames = []
        
        patch_kw = dict(patch_size=5,      # 5x5 patches
                        patch_distance=6,  # 13x13 search area
                        multichannel=True)
        
        for index, picture in X.iterrows():
            img = picture.Picture
            prepfilename = picture.prepfilename
            
            temp_img, temp_sift, temp_color = self.treat_pictures(img, prepfilename)
            colors.append(temp_color)
            sift_desc.append(temp_sift)
        
        print('Pictures loaded')
        
        return sift_desc, colors
    
    def transform(self, X, y=None):
        sift_desc, colors = self.load_pics(X, y)
        bow = self.compute_data(sift_desc, colors)
        Xbow = self.transform_bow(bow)
        
        return Xbow
    
    def fit_sift(self, descriptors):
        indices = np.random.randint(0,descriptors.shape[0],self.clf_params['kmean_sample_size'])
        Xsample = descriptors[indices]
        Xscaled = self.scaler.fit_transform(Xsample)
        if self.clf_params['apply_pca']:
            Xscaled = self.pca.fit_transform(Xscaled)
            print('PCA fitted')
        else:
            print('No PCA')
        self.kmeans.fit(Xscaled)
        print('Sift fitted')
        return self
    
    def compute_data(self, descriptors, colors):
        toreturn = []
        for descriptor , color in zip(descriptors, colors):
            #Key points treatment
            XScaled = self.scaler.transform(descriptor)
            if self.clf_params['apply_pca']:
                XScaled = self.pca.transform(XScaled)
            bow_keys = [self.kmeans.predict(keydescription.reshape(1, -1))[0] for keydescription in XScaled]
            keystring = ' '.join(['kp'+str(x) for x in bow_keys])
            
            #Colors treatment
            XScaled = self.color_scaler.transform(color)
            bow_colors = [self.color_kmeans.predict(majorcolor.reshape(1, -1))[0] for majorcolor in XScaled]
            colorstring = ' '.join(['cl'+str(x) for x in bow_colors])
            
            toreturn.append(keystring+' '+colorstring) #
        return toreturn
    
    def exportset(self, X):
        sift_desc, colors = self.load_pics(X)
        bow = self.compute_data(sift_desc, colors)
        Xbow = self.transform_bow(bow)
        return Xbow
    
    def treat_pictures(self, picture, prepfilename='zzz'):

        imcv = img_as_ubyte(picture)
        color_dominance = self.color_domincance(imcv)
        imcv_gr = cv2.cvtColor(imcv, cv2.COLOR_BGR2GRAY)
        
        #print(imcv.shape)
        
        #plt.imshow(imcv_gr, cmap='gray')
        #plt.axis("off")
        #plt.show()
        
        imcv_kp, imcv_desc = self.sift.detectAndCompute(imcv_gr, None)

        if self.pics_params['save_pics']:
            #Image saving OpenCV
            cv2.imwrite('../pics/{}.jpg'.format(prepfilename),cv2.drawKeypoints(imcv, imcv_kp, imcv,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
        
        return picture, imcv_desc, color_dominance

dogreconizer = SiftBOW(**{'pics_save_pics':False, 'clf_apply_pca':False})
dogreconizer.get_params()
dogreconizer.fit(X_train, y_train)

/Users/cricket/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/Users/cricket/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
/Users/cricket/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


Pictures loaded


ValueError: all the input arrays must have same number of dimensions

In [ ]:
dogreconizer.get_params()
Xbowtrain = dogreconizer.transform(X_train)
Xbowtest = dogreconizer.transform(X_test)

In [ ]:
np.savez('bowsift', Xbowtrain=Xbowtrain.toarray(),
                      Xbowtest=Xbowtest.toarray(),
                      y_test=y_test,
                      y_train=y_train)

In [15]:
zz = np.load('bowsift.npz')

In [ ]:
zz['Xbowtrain']
zz.files

In [ ]:
zz['y_train']

# Pipeline creation

In [ ]:
dogreconizer = SiftBOW(**{'pics_save_pics':True, 'clf_apply_pca':False})
clf = RandomForestClassifier()
pipe = [('bow', dogreconizer), ('clf',clf)]

dogpipe = Pipeline(pipe)

In [ ]:
dogpipe.fit(X_train, y_train)

In [ ]:
dogpipe.score(X_test, y_test)

# gridsearch

In [ ]:
parameters = {'bow__color_n_clusters':[30,50], 'bow__kmeans_n_clusters':[30,50], 'clf__n_estimators':[100,200], 'clf__max_depth':[50, None]}

dogclf = GridSearchCV(dogpipe, parameters, n_jobs=3, cv=3, verbose=5)

dogclf.fit(X_train, y_train)




In [ ]:
dogpipe.get_params()

In [ ]:
y_test

In [ ]:
clf = RandomForestClassifier()
dogreconizer = SiftBOW(clf, **{'pics_save_pics':True, 'clf_apply_pca':False})

In [ ]:
XX_train = dogreconizer.fit(X_train, y_train)

In [ ]:
y_pred = dogreconizer.predict(X_test, y_test)
y_pred

In [ ]:
confusion_matrix(y_test, y_pred)


In [ ]:
dogreconizer.score(X_test, y_test)

In [ ]:
XX_test = dogreconizer.exportset(X_test)

In [ ]:
X_train.shape

# KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=12, n_jobs=3)
neigh.fit(XX_train.toarray(), y_train.values)
neigh.score(XX_test.toarray(), y_test.values)

In [ ]:
parameters = {'n_neighbors':[3,5,7,9,15,20], 'leaf_size': [10,20,30,40,50]}
scorer = make_scorer(f1_score, average='micro')

clfknn = GridSearchCV(neigh, parameters, verbose=1, cv=3, n_jobs=5, scoring=scorer)
clfknn.fit(XX_train.toarray(), y_train.values)
clfknn.score(XX_test.toarray(), y_test.values)

# SVC

In [ ]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(XX_train.toarray(), y_train.values)
clf.score(XX_test.toarray(), y_test.values)

In [ ]:
parameters = {'C':np.logspace(-5, 5, 11), 'gamma':np.logspace(-5, 5, 11)}
scorer = make_scorer(f1_score, average='micro')

clfsvc = GridSearchCV(clf, parameters, verbose=1, cv=3, n_jobs=5, scoring=scorer)
clfsvc.fit(XX_train.toarray(), y_train.values)
clfsvc.score(XX_test.toarray(), y_test.values)

In [ ]:
clfsvc.best_params_

# Random Forrest

In [ ]:
clf = RandomForestClassifier(random_state=0)

In [ ]:
parameters = {'n_estimators':[10,50,100,500],
              'max_depth':[None,10,15,20],
              'min_samples_split':[2,3,5],}
scorer = make_scorer(f1_score, average='micro')

clfrfc = GridSearchCV(clf, parameters, verbose=1, cv=5, n_jobs=3, scoring=scorer)
clfrfc.fit(XX_train.toarray(), y_train.values)
clfrfc.score(XX_test.toarray(), y_test.values)

In [ ]:
clfrfc.best_params_

In [ ]:
clf = RandomForestClassifier(random_state=0, **clfrfc.best_params_)
clf.fit(XX_train.toarray(), y_train.values)
clf.feature_importances_




# GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()

In [ ]:
parameters = {'n_estimators':[10,50,100,500],
              'max_depth':[None,10,15,20],
              'min_samples_split':[2,3,5],}
scorer = make_scorer(f1_score, average='micro')

clfgbc = GridSearchCV(clf, parameters, verbose=1, cv=5, n_jobs=3, scoring=scorer)
clfgbc.fit(XX_train.toarray(), y_train.values)
clfgbc.score(XX_test.toarray(), y_test.values)

In [ ]:
clfgbc.best_params_

In [ ]:
clf = GradientBoostingClassifier(**clfgbc.best_params_)
clf.fit(XX_train.toarray(), y_train.values)

In [ ]:
# ZCA whitening
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from keras import backend as K
K.set_image_dim_ordering('th')
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
# convert from int to float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# define data preparation
datagen = ImageDataGenerator(zca_whitening=True)
# fit parameters from data
datagen.fit(X_train)
# configure batch size and retrieve one batch of images
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=9):
    # create a grid of 3x3 images
    for i in range(0, 9):
        pyplot.subplot(330 + 1 + i)
        pyplot.imshow(X_batch[i].reshape(28, 28), cmap=pyplot.get_cmap('gray'))
    # show the plot
    pyplot.show()
    break

In [ ]:
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape

In [ ]:
plt.imshow(X_train[1].reshape(28, 28), cmap=pyplot.get_cmap('gray'))
plt.show()

In [ ]:
X_train[0][0]

In [ ]:
class SiftBOW(BaseEstimator, TransformerMixin):

    def __init__(self, **params):        
        self.kmeans_params = {'n_clusters':30, 'verbose':0, 'random_state':3, 'max_iter': 300}
        self.sift_params = {'nfeatures':0}
        self.pca_params = {'n_components':10, 'random_state':0}
        self.cvect_params = {'ngram_range':(1,1), 'min_df':1, 'max_df':1.}
        self.color_params = {'n_clusters':30, 'random_state':0}
        self.pics_params = {'save_pics':True, 'equalyze':True, }
        
        self.clf_params = { 'kmean_sample_size':10000}
        
        self.params = {**{'kmeans_'+k:v for k,v in self.kmeans_params.items()},
                       **{'sift_'+k:v for k,v in self.sift_params.items()},
                       **{'cvect_'+k:v for k,v in self.cvect_params.items()},
                       **{'clf_'+k:v for k,v in self.clf_params.items()},
                       **{'pca_'+k:v for k,v in self.pca_params.items()},
                       **{'color_'+k:v for k,v in self.color_params.items()},
                       **{'pics_'+k:v for k,v in self.pics_params.items()},
                       **params}
        
        self.update_params()
        self.kmeans = KMeans(**self.kmeans_params)
        self.scaler = StandardScaler()
        self.color_scaler = StandardScaler()
        self.cvect = CountVectorizer(**self.cvect_params, analyzer='word', tokenizer=None)
        self.sift = cv2.xfeatures2d.SIFT_create(**self.sift_params)
        #self.sift = cv2.ORB()
        self.pca = PCA(**self.pca_params)
        self.color_kmeans = KMeans(**self.color_params)
        
    def get_params(self, deep=True):
        return self.params
    
    def set_params(self, **params):
        self.params = {**self.params, **params}
        self.update_params()
        return self
        
    def update_params(self):
        
        self.kmeans_params = {k[7:]:v for k,v in self.params.items() if k.startswith('kmeans_')}
        self.sift_params = {k[5:]:v for k,v in self.params.items() if k.startswith('sift_')}
        self.pics_params = {k[5:]:v for k,v in self.params.items() if k.startswith('pics_')}
        self.color_params = {k[6:]:v for k,v in self.params.items() if k.startswith('color_')}
        self.cvect_params = {k[6:]:v for k,v in self.params.items() if k.startswith('cvect_')}
        self.pca_params = {k[4:]:v for k,v in self.params.items() if k.startswith('pca_')}
        self.clf_params = {k[4:]:v for k,v in self.params.items() if k.startswith('clf_')}

        return self

    
    def fit_bow(self, keydescriptors):
        self.cvect.fit(keydescriptors)
        print('BOW fitted')
        return self
    
    def fit_colors(self, colors, samplesize=10000):
        colorlist = colors.reshape(-1,3)
        indices = np.random.randint(0, colorlist.shape[0], samplesize)
        Xsample = colors[indices]
        Xscaled = self.color_scaler.fit_transform(Xsample)
        self.color_kmeans.fit(Xscaled)
        print('Color dominance fitted')
        return self
    
    def transform_bow(self, keydescriptors):
        return self.cvect.transform(keydescriptors)
    
    def fit(self, X, y= None):
        pics, sift_desc, color_dominance = self.transform(X, y)
        self.fit_sift(np.concatenate(sift_desc))
        self.fit_colors(np.concatenate(color_dominance))
        
        bow = self.compute_data(sift_desc, color_dominance)
        self.fit_bow(bow)
        Xbow = self.transform_bow(bow)
        return Xbow
    
    def color_domincance(self, picture, n_cluster=10):
        #Converts to openCV
        imcv = cv2.cvtColor(np.array(picture), cv2.COLOR_RGB2BGR)
        
        #Flatten all the pixels
        Z = imcv.reshape((-1,3))

        # convert to np.float32
        Z = np.float32(Z)

        # define criteria, number of clusters(K) and apply kmeans()
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        K = n_cluster
        ret, label, color_dominance = cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
        return color_dominance
    
    def transform(self, X, y=None):
        sift_desc = []
        colors = []
        pics = []
        filenames = []
        
        patch_kw = dict(patch_size=5,      # 5x5 patches
                        patch_distance=6,  # 13x13 search area
                        multichannel=True)
        
        for index, picture in X.iterrows():
            xmax = picture.xmax
            ymax = picture.ymax
            xmin = picture.xmin
            ymin = picture.ymin
            foldername = picture.foldername
            breed = picture.breed
            filename = picture.filename
            prepfilename = picture.prepfilename
        
            filenames.append(prepfilename)
        
            #Image opening
            img = Image.open('../Images/n{}-{}/{}.jpg'.format(foldername, breed, filename))
            
            color_dominance = self.color_domincance(img)
            
            #Image cropping
            area = (xmin, ymin, xmax, ymax)
            img = img.crop(area)
            
            #Image equalization
            #img = ImageOps.equalize(img)
            img = exposure.equalize_hist(np.array(img))
            
            #Gaussian blur fliter
            sigma_est = np.mean(estimate_sigma(np.array(img), multichannel=True, average_sigmas=False))
            #print(sigma_est)
            #img = img.filter(ImageFilter.GaussianBlur(radius=sigma_est))
            #img = gaussian(np.array(img), sigma=sigma_est, multichannel=True)
            
            #Non linear mean
            img = denoise_nl_means(img, h=1.5*sigma_est, fast_mode=True, **patch_kw)
            

            
            temp_img, temp_sift, temp_color = self.treat_pictures(img, prepfilename)
            colors.append(temp_color)
            pics.append(temp_img)
            sift_desc.append(temp_sift)
        
        print('Pictures loaded')
        
        return pics, sift_desc, colors
    
    def predict(self, X, y):
        pics, sift_desc, colors = self.transform(X, y)
        bow = self.compute_data(sift_desc, colors)
        Xbow = self.transform_bow(bow)
        
        return self.estimator.predict(Xbow)
    
    def fit_sift(self, descriptors):
        indices = np.random.randint(0,descriptors.shape[0],self.clf_params['kmean_sample_size'])
        Xsample = descriptors[indices]
        Xscaled = self.scaler.fit_transform(Xsample)
        if self.clf_params['apply_pca']:
            Xscaled = self.pca.fit_transform(Xscaled)
            print('PCA fitted')
        else:
            print('No PCA')
        self.kmeans.fit(Xscaled)
        print('Sift fitted')
        return self
    
    def compute_data(self, descriptors, colors):
        toreturn = []
        for descriptor , color in zip(descriptors, colors):
            #Key points treatment
            XScaled = self.scaler.transform(descriptor)
            if self.clf_params['apply_pca']:
                XScaled = self.pca.transform(XScaled)
            bow_keys = [self.kmeans.predict(keydescription.reshape(1, -1))[0] for keydescription in XScaled]
            keystring = ' '.join(['kp'+str(x) for x in bow_keys])
            
            #Colors treatment
            XScaled = self.color_scaler.transform(color)
            bow_colors = [self.color_kmeans.predict(majorcolor.reshape(1, -1))[0] for majorcolor in XScaled]
            colorstring = ' '.join(['cl'+str(x) for x in bow_colors])
            
            toreturn.append(keystring+' '+colorstring) #
        return toreturn
    
    def exportset(self, X):
        pics, sift_desc, colors = self.transform(X)
        bow = self.compute_data(sift_desc, colors)
        Xbow = self.transform_bow(bow)
        return Xbow
    
    def treat_pictures(self, picture, prepfilename='zzz'):

        imcv = img_as_ubyte(picture)
        color_dominance = self.color_domincance(imcv)
        imcv_gr = cv2.cvtColor(imcv, cv2.COLOR_BGR2GRAY)
        
        print(imcv.shape)
        
        plt.imshow(imcv_gr, cmap='gray')
        plt.axis("off")
        plt.show()
        
        imcv_kp, imcv_desc = self.sift.detectAndCompute(imcv_gr, None)

        if self.pics_params['save_pics']:
            #Image saving OpenCV
            cv2.imwrite('../pics/{}.jpg'.format(prepfilename),cv2.drawKeypoints(imcv, imcv_kp, imcv,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
        
        return picture, imcv_desc, color_dominance

dogreconizer = SiftBOW(**{'pics_save_pics':True, 'clf_apply_pca':False})
dogreconizer.get_params()
dogreconizer.fit(X_train[:5], y_train[:5])